# The mirroring is quite computational expensive therefore the diagonal blocks get precomputed

In [1]:
import cloudpickle
from scipy.sparse import dok_matrix
from scipy.sparse import tril
import scipy

def load_dict(path,zeilen,spalten):
    if(spalten > zeilen):
        template = "block_{i}_{j}".format(i=spalten,j=zeilen)
    else:
        template = "block_{i}_{j}".format(i=zeilen,j=spalten)

    file_path = path + '\\' + template
    with open(file_path, 'rb+') as file:
        co_occurences = cloudpickle.load(file)
    
    return co_occurences


def load_co_occurence(path,zeilen,spalten):
    co_occurences = load_dict(path,zeilen,spalten)
    coocurrence = dok_matrix((20000,20000),dtype='d')
    
    coocurrence._update(co_occurences) # dok_matrix updates #7673 pull request

    if spalten > zeilen :
        raise Exception("You sha'll not get symmetrical context")
        #may need to fix this in the future due to other non symmetrical feature
    
    if spalten == zeilen:
        print('mirroring')
        #print(coocurrence.toarray())
        coocurrence = coocurrence + tril(coocurrence,k=-1).transpose()
    return coocurrence

In [2]:
import re
import os
import h5py
import numpy as np
import tensorflow as tf

# The default block length of 20000 must be divisible by this number. 
split_length = 4 #2,4,5,10
size = int(20000/split_length)

path = 'S:\\tmp\\coocurrence_blocks'
path_hdf = r'S:\\base_coocurrence_hdf_'+str(size)+'\\'
regex = r'block_([0-9]{1,})_([0-9]{1,})'

for file_name in os.listdir(path):
    match = re.match(regex, file_name)
    i,j   = match.group(1), match.group(2)
    print('output for progress ' + str(i)+','+str(j))
    if(i < j):
        continue
    
    
    co_occurence = load_co_occurence(path,i,j)
    co_occurence  = co_occurence.toarray()
    if(i == j):
        np.fill_diagonal(co_occurence,0)
    
    for sub_i in range(split_length):
        for sub_j in range(split_length):
            a,b = int(i)*split_length+sub_i , int(j)*split_length+sub_j
            filename = 'tf_cooccurence_{a}_{b}.tensor'.format(a = a,b = b)
            tf_subcur = tf.convert_to_tensor(co_occurence[sub_i*size:(sub_i+1)*size,sub_j*size:(sub_j+1)*size],dtype=tf.float32)
            #hdf_file = h5py.File( path_hdf + filename, "w")
            #co_occurence_hdf = hdf_file.create_dataset("co-ocurrence", (size, size))
            #co_occurence_hdf[:,:] = co_occurence[sub_i*size:(sub_i+1)*size,sub_j*size:(sub_j+1)*size]
            #hdf_file.close()
            with open(path_hdf + filename, 'wb+') as file:
                cloudpickle.dump(tf_subcur, file)
        
    #print(co_occurence_hdf)

output for progress 0,0
mirroring
output for progress 0,1
output for progress 0,2
output for progress 0,3
output for progress 0,4
output for progress 0,5
output for progress 0,6
output for progress 0,7
output for progress 1,0
output for progress 1,1
mirroring
output for progress 1,2
output for progress 1,3
output for progress 1,4
output for progress 1,5
output for progress 1,6
output for progress 1,7
output for progress 2,0
output for progress 2,1
output for progress 2,2
mirroring
output for progress 2,3
output for progress 2,4
output for progress 2,5
output for progress 2,6
output for progress 2,7
output for progress 3,0
output for progress 3,1
output for progress 3,2
output for progress 3,3
mirroring
output for progress 3,4
output for progress 3,5
output for progress 3,6
output for progress 3,7
output for progress 4,0
output for progress 4,1
output for progress 4,2
output for progress 4,3
output for progress 4,4
mirroring
output for progress 4,5
output for progress 4,6
output for pro